In [1]:
import pandas as pd

TRANSFORMACIONES

In [2]:
# Importo los datos de las plataformas
amazon = pd.read_csv('amazon_prime_titles.csv')
disney = pd.read_csv('disney_plus_titles.csv')
hulu = pd.read_csv('hulu_titles.csv')
netflix = pd.read_csv('netflix_titles.csv')

In [6]:
# Genero campo id
amazon["id"] = "a" + amazon["show_id"]
disney["id"] = "d" + disney["show_id"]
hulu["id"] = "h" + hulu["show_id"]
netflix["id"] = "n" + netflix["show_id"]

In [30]:
# Armo un solo df para realizar el resto de las transformaciones (para realizarlas una sola vez y no cuatro)
bbdd_plat = pd.concat([amazon, disney, hulu, netflix], ignore_index=True)

In [31]:
# Reorganizo las columnas de bbdd_plataformas
bbdd_plat = bbdd_plat.reindex(columns=['id','show_id','type','title','director','cast','country','date_added','release_year','rating','duration','listed_in','description'])


In [32]:
# Reemplazo los campos nulos de la columna 'rating' por 'g'
bbdd_plat['rating'].fillna("G", inplace=True)

In [33]:
# Cambio formato de fechas a AAAA-mm-dd
bbdd_plat['date_added'] = pd.to_datetime(bbdd_plat['date_added'])

In [34]:
# Expreso los campos de texto en minúscula
bbdd_plat['show_id'] = bbdd_plat['show_id'].str.lower()
bbdd_plat['type'] = bbdd_plat['type'].str.lower()
bbdd_plat['title'] = bbdd_plat['title'].str.lower()
bbdd_plat['director'] = bbdd_plat['director'].str.lower()
bbdd_plat['cast'] = bbdd_plat['cast'].str.lower()
bbdd_plat['country'] = bbdd_plat['country'].str.lower()
bbdd_plat['rating'] = bbdd_plat['rating'].str.lower()
bbdd_plat['duration'] = bbdd_plat['duration'].str.lower()
bbdd_plat['listed_in'] = bbdd_plat['listed_in'].str.lower()
bbdd_plat['description'] = bbdd_plat['description'].str.lower()

In [35]:
# Divido el campo duration en dos campos: duration_int (int) y duration_type (str)
bbdd_plat[['duration_int', 'duration_type']] = bbdd_plat['duration'].str.split(' ', 1, expand=True)

C:\Users\Daniela\AppData\Local\Temp\ipykernel_2884\406192419.py:2: FutureWarning: In a future version of pandas all arguments of StringMethods.split except for the argument 'pat' will be keyword-only.
  bbdd_plat[['duration_int', 'duration_type']] = bbdd_plat['duration'].str.split(' ', 1, expand=True)


In [25]:
# Ahora trabajo con los datos de 'Rating'
rating1 = pd.read_csv('1.csv')
rating2 = pd.read_csv('2.csv')
rating3 = pd.read_csv('3.csv')
rating4 = pd.read_csv('4.csv')
rating5 = pd.read_csv('5.csv')
rating6 = pd.read_csv('6.csv')
rating7 = pd.read_csv('7.csv')
rating8 = pd.read_csv('8.csv')

In [26]:
# Uno todos los 'ratings' en un solo df
ratings_compl = pd.concat([rating1, rating2, rating3, rating4, rating5, rating6, rating7, rating8], ignore_index=True)

In [36]:
# Cambio el nombre de la columna movieId por id para poder unirla a bbdd_plataformas, rating por score, 
ratings_compl.rename(columns={'movieId': 'id'}, inplace=True)
ratings_compl.rename(columns={'rating': 'score'}, inplace=True)

In [37]:
# Agrupo los score según id
rating_promedio = ratings_compl.groupby('id').mean()

In [38]:
# Uno los dataframes bbdd_plataformas y ratings:
bbdd_plat_compl = pd.merge(bbdd_plat,rating_promedio, on=["id"])

In [40]:
# Elimino las comumnas duration, user_id y timestamp que son innecesarias para el estudio
bbdd_plat_compl.drop(columns=['duration', 'userId', 'timestamp'], inplace=True)

In [41]:
# Exporto la bbdd
bbdd_plat_compl.to_csv("bbdd_plataformas_completo.csv", sep=';')

Realizo las funciones para las consultas en la API

In [42]:
# •	Película con mayor duración con filtros opcionales de AÑO, PLATAFORMA Y TIPO DE DURACIÓN. (la función debe llamarse 
# get_max_duration(year, platform, duration_type))

def get_max_duration(year, platform, duration_type):
    peliculas = []
    peli_mayor = []
    for indice, x in enumerate (bbdd_plat_compl['release_year']):
        if x == year:
            if bbdd_plat_compl['id'][indice][0] == platform:
                if bbdd_plat_compl['duration_type'][indice] == duration_type:
                    peliculas.append(int(bbdd_plat_compl['duration_int'][indice]))
                    peli_mayor.append(indice)
                    mayor_time = max(peliculas)
                    peli = peliculas.index(mayor_time)
                    respuesta = bbdd_plat_compl['title'][peli_mayor[peli]]

    return respuesta

In [43]:
print(get_max_duration(2020, 'd', 'min'))

dory's reef cam


In [44]:
# •	Cantidad de películas por plataforma con un puntaje mayor a XX en determinado año 
# (la función debe llamarse get_score_count(platform, scored, year))

def get_score_count(platform, scored, year):
    puntaje_mayor = []
    for indice, x in enumerate(bbdd_plat_compl['release_year']):
        if x == year:
            if bbdd_plat_compl['id'][indice][0] == platform:
                if bbdd_plat_compl['score'][indice] > scored:
                    puntaje_mayor.append(x)
    
    return len(puntaje_mayor)

In [45]:
print(get_score_count('d', 3.5, 2020))

81


In [46]:
# •	Cantidad de películas por plataforma con filtro de PLATAFORMA. 
# (La función debe llamarse get_count_platform(platform))

def get_count_platform(platform):
    pelis_plataforma = []
    for indice, x in enumerate(bbdd_plat_compl['id']):
        if bbdd_plat_compl['id'][indice][0] == platform:
            pelis_plataforma.append(x)
    
    return len(pelis_plataforma) 
    

In [47]:
print(get_count_platform('d'))

1450


In [89]:
# •	Actor que más se repite según plataforma y año. 
# (La función debe llamarse get_actor(platform, year))

def get_actor(platform, year):
    filtro = []
    contador = 0
    
    for indice, x in enumerate(bbdd_plat_compl['release_year']):
        if x == year:
            if bbdd_plat_compl['id'][indice][0] == platform:
                filtro.append(bbdd_plat_compl['cast'][indice])
    
    for ind, z in enumerate(filtro):
        if type(z) == str:
            actores = filtro[ind].split(',')
    
    for ind, z in enumerate(actores):
        if actores.count(z) > contador:
            contador = actores.count(z)
            actor_mayor = actores[ind]

    return actor_mayor

In [90]:
print(get_actor('d', 2021))

elizabeth olsen


In [91]:
print(get_actor('a', 2020))

clara kovacic
